In [52]:
import json
import pandas as pd

# Load the JSON data

json_file_path = r'C:\Users\mueller15\OneDrive - Avaya\AXP Analytics\DialogFlow\bquxjob_1badbb0_187b38bd236.json'

with open(json_file_path, 'r') as j:
    data = json.loads(j.read())


# Create an empty list to hold the parsed data
parsed_data = []

# Loop through each conversation turn and extract the relevant information
for turn in data:
    conversation_name = turn['conversation_name']
    turn_position = turn['turn_position']
    request_time = turn['request_time']
    language_code = turn['language_code']
    request = json.loads(turn['request'])
    response = json.loads(turn['response'])
    derived_data = json.loads(turn['derived_data'])
    conversation_signals = json.loads(turn['conversation_signals'])

    # Create a dictionary with the parsed data for this conversation turn
    parsed_turn = {
        'conversation_name': conversation_name,
        'turn_position': turn_position,
        'request_time': request_time,
        'language_code': language_code,
        'request': request,
        'response': response,
        'derived_data': derived_data,
        'conversation_signals': conversation_signals
    }

    # Append the parsed data for this conversation turn to the list of parsed data
    parsed_data.append(parsed_turn)

# Create a pandas DataFrame from the parsed data, with a unique identifier column
df = pd.DataFrame(parsed_data)
df['unique_identifier'] = df['conversation_name'] + '_' + df['turn_position']

def flatten_json_columns(df):
    """
    Flatten all columns that contain JSON data into new columns.
    """
    json_columns = []
    for column in df.columns:
        if df[column].apply(lambda x: isinstance(x, (dict, list))).all():
            json_columns.append(column)

    for column in json_columns:
        column_data = df[column].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
        flattened_data = pd.json_normalize(column_data)
        flattened_data.columns = [f"{column}_{col}" for col in flattened_data.columns]
        df = pd.concat([df, flattened_data], axis=1)
        df = df.drop(column, axis=1)

    return df
df = flatten_json_columns(df)


In [53]:
df.describe()
df

,conversation_name,turn_position,request_time,language_code,unique_identifier,request_debugMode,request_session,request_queryInput.languageCode,request_queryInput.text.text,request_queryParams.timeZone,...,derived_data_page.displayName,derived_data_page.name,derived_data_webhookPayloads,derived_data_webhookStatuses,conversation_signals_turnSignals.reachedEndPage,conversation_signals_turnSignals.webhookStatuses,conversation_signals_turnSignals.failureReasons,conversation_signals_turnSignals.agentEscalated,conversation_signals_turnSignals.noUserInput,conversation_signals_turnSignals.noMatch
0,projects/ccai-av-069d8009a01576/locations/euro...,3,2023-03-22 17:27:42.897124 UTC,en,projects/ccai-av-069d8009a01576/locations/euro...,True,projects/ccai-av-069d8009a01576/locations/euro...,en,bailout,Europe/Madrid,...,End Session,projects/ccai-av-069d8009a01576/locations/euro...,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN
1,projects/ccai-av-069d8009a01576/locations/euro...,4,2023-03-17 08:39:43.082446 UTC,en,projects/ccai-av-069d8009a01576/locations/euro...,True,projects/ccai-av-069d8009a01576/locations/euro...,en,bailout,Europe/Madrid,...,End Session,projects/ccai-av-069d8009a01576/locations/euro...,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN
2,projects/ccai-av-069d8009a01576/locations/euro...,3,2023-03-17 08:39:20.291382 UTC,en,projects/ccai-av-069d8009a01576/locations/euro...,True,projects/ccai-av-069d8009a01576/locations/euro...,en,Can you tell me something about primary colors,Europe/Madrid,...,Start Page,projects/ccai-av-069d8009a01576/locations/euro...,[{}],[{}],NaN,[OK],NaN,NaN,NaN,NaN
3,projects/ccai-av-069d8009a01576/locations/euro...,1,2023-03-22 17:23:01.661526 UTC,en,projects/ccai-av-069d8009a01576/locations/euro...,True,projects/ccai-av-069d8009a01576/locations/euro...,en,Hallo,Europe/Madrid,...,Start Page,projects/ccai-av-069d8009a01576/locations/euro...,[{}],"[{'code': 14, 'message': 'State: URL_UNREACHAB...",NaN,[UNAVAILABLE],[FAILED_WEBHOOK],NaN,NaN,NaN
4,projects/ccai-av-069d8009a01576/locations/euro...,1,2023-03-21 17:20:36.685650 UTC,en,projects/ccai-av-069d8009a01576/locations/euro...,True,projects/ccai-av-069d8009a01576/locations/euro...,en,Hello,Europe/Madrid,...,Start Page,projects/ccai-av-069d8009a01576/locations/euro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,projects/ccai-av-069d8009a01576/locations/euro...,3,2023-04-19 12:36:30.455460 UTC,en,projects/ccai-av-069d8009a01576/locations/euro...,NaN,projects/ccai-av-069d8009a01576/locations/euro...,en-us,New Order,NaN,...,New Order,projects/ccai-av-069d8009a01576/locations/euro...,[{}],"[{'code': 5, 'message': 'Error calling webhook...",NaN,[NOT_FOUND],[FAILED_WEBHOOK],NaN,NaN,NaN
577,projects/ccai-av-069d8009a01576/locations/euro...,4,2023-04-19 12:36:34.918345 UTC,en,projects/ccai-av-069d8009a01576/locations/euro...,NaN,projects/ccai-av-069d8009a01576/locations/euro...,en-us,NaN,NaN,...,New Order,projects/ccai-av-069d8009a01576/locations/euro...,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN
578,projects/ccai-av-069d8009a01576/locations/euro...,5,2023-04-19 12:36:40.063927 UTC,en,projects/ccai-av-069d8009a01576/locations/euro...,NaN,projects/ccai-av-069d8009a01576/locations/euro...,en-us,happy yellow,NaN,...,New Order,projects/ccai-av-069d8009a01576/locations/euro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
579,projects/ccai-av-069d8009a01576/locations/euro...,6,2023-04-19 12:36:44.792987 UTC,en,projects/ccai-av-069d8009a01576/locations/euro...,NaN,projects/ccai-av-069d8009a01576/locations/euro...,en-us,NaN,NaN,...,New Order,projects/ccai-av-069d8009a01576/locations/euro...,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN
